In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,347 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,372 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 5,060 kB i

In [ ]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

findspark.init()

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [8]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DateType, StringType, IntegerType
from pyspark.sql.functions import spark_partition_id

config = SparkConf().setMaster('local').setAppName('lab22')
spark = SparkSession.builder.config(conf=config).getOrCreate()
sc = spark.sparkContext

DATASET_PATH = '/content/drive/MyDrive/sample.csv'

HELLO SPARK SQL

In [ ]:
survey_schema_struct = StructType([
    StructField('Timestamp', StringType()),
    StructField('Age', IntegerType()),
    StructField('Gender', StringType()),
    StructField('Country', StringType()),
    StructField('state', StringType()),
    StructField('self_employed', StringType()),
    StructField('family_history', StringType()),
    StructField('treatment', StringType()),
    StructField('work_interfere', StringType()),
    StructField('no_employees', StringType()),
    StructField('remote_work', StringType()),
    StructField('tech_company', StringType()),
    StructField('benefits', StringType()),
    StructField('care_options', StringType()),
    StructField('wellness_program', StringType()),
    StructField('seek_help', StringType()),
    StructField('anonymity', StringType()),
    StructField('leave', StringType()),
    StructField('mental_health_consequence', StringType()),
    StructField('phys_health_consequence', StringType()),
    StructField('coworkers', StringType()),
    StructField('supervisor', StringType()),
    StructField('mental_health_interview', StringType()),
    StructField('phys_health_interview', StringType()),
    StructField('mental_vs_physical', StringType()),
    StructField('obs_consequence', StringType()),
    StructField('comments', StringType())
])
flight_time_csv_DF = spark.read.format('csv').option('header','true').option('mode','FAILFAST').option('dateFormat', 'M/d/y').schema(flight_schema_struct).load(DATASET_PATH)

In [11]:
surveyDF = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(DATASET_PATH)

# Create temp view
surveyDF.createOrReplaceTempView('Employee_tbl')
# Run query
countDF = spark.sql('SELECT e.Country, count(*) FROM Employee_tbl e WHERE e.Age < 40 GROUP BY e.Country')

countDF.show()

+--------------+--------+
|       Country|count(1)|
+--------------+--------+
| United States|       4|
|        Canada|       2|
|United Kingdom|       1|
+--------------+--------+

